In [ ]:
import tensorflow as tf
tf.keras.backend.clear_session()

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
pip install python-Levenshtein

     |████████████████████████████████| 51kB 6.9MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144790 sha256=4ae5e562de8e48c79ddabd359623ae3e0e417aff282f7c9c908def0502e01b10
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein


In [ ]:
import librosa
import numpy as np
import pdb
import string
import pandas as pd
from Levenshtein import distance

In [ ]:
def wavs2feat(wavfiles):
    '''
    Concatenate the audio files listed in wavfiles
    Input: list of audio wav file names
    Output: Magnitude spectrogram of concatenated wav
    '''
    x = []

    for wf in wavfiles:
        x1, Fs = librosa.load(wf, sr=44100, mono=True)
        x.append(x1)

    hop = int(0.01 * Fs) # 10ms
    win = int(0.02 * Fs) # 20ms
    X = np.hstack(x)
    X = librosa.stft(X, n_fft=1024, hop_length=hop, win_length=win, window='hann', center=True, pad_mode='reflect')
    X = np.abs(X)
    X_db = process(X)
    return X_db

def read_csv(filename):
    id_label = {}
    with open(filename,'r') as fid:
        for line in fid: # '176787-5-0-27.wav,engine_idling\n'
            tokens = line.strip().split(',') # ['176787-5-0-27.wav', 'engine_idling']
            id_label[tokens[0]] = tokens[1]
    return id_label

def process(X):

    if(X.shape[1]<2001):
      pad_size = 2001 - X.shape[1]
      zero_x = np.zeros((X.shape[0],pad_size))
      X = np.hstack((X,zero_x))

    n_mels = 128
    mel = librosa.filters.mel(sr=44100, n_fft=1024, n_mels=n_mels)
    X_db = mel.dot(X)
    return X_db

In [ ]:
import numpy as np
import random
import pandas as pd

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/project_specs_698/shared_train/labels_train.csv")
Labels = list(df['class'].unique())
Labels_int = list(range(len(Labels)))
map_char_to_int = dict(zip(Labels,Labels_int))
print(map_char_to_int)
a = [0]*10
for i in df['class']:
  a[map_char_to_int[i]]+=1
print(a)

{'dog_bark': 0, 'gun_shot': 1, 'engine_idling': 2, 'siren': 3, 'jackhammer': 4, 'drilling': 5, 'children_playing': 6, 'street_music': 7, 'air_conditioner': 8, 'car_horn': 9}
[200, 70, 196, 177, 240, 200, 200, 200, 200, 78]


In [ ]:
def encode(labels_lst,channel_size,NUM_CLASSES):
    y = np.zeros((NUM_CLASSES+1,channel_size))#one extra class for silence
    curr_channel = 0
    for i in range(len(labels_lst)):
      y[labels_lst[i]][curr_channel] = 1
      curr_channel = curr_channel+1

    if (len(labels_lst)<channel_size):
      temp = channel_size - len(labels_lst)
      for i in range(len(labels_lst),channel_size):
        y[NUM_CLASSES][i] = 1



    return y.T  


In [ ]:
id_label = read_csv("/content/drive/MyDrive/project_specs_698/shared_train/labels_train.csv")

X = [[] for i in range(10)]

for key,value in id_label.items():
  if(value=='dog_bark'):
    X[0].append(key)
    random.shuffle(X[0])

  elif(value=='gun_shot'):
    X[1].append(key)
    random.shuffle(X[1])

  elif(value=='engine_idling'):
    X[2].append(key)
    random.shuffle(X[2])

  elif(value=='siren'):
    X[3].append(key)
    random.shuffle(X[3])

  elif(value=='jackhammer') :
    X[4].append(key)
    random.shuffle(X[4])

  elif(value=='drilling'):
    X[5].append(key)
    random.shuffle(X[5])

  elif(value=='children_playing'):
    X[6].append(key)
    random.shuffle(X[6])

  elif(value=='street_music'):
    X[7].append(key)
    random.shuffle(X[7])

  elif(value=='air_conditioner'):
    X[8].append(key)
    random.shuffle(X[8])

  elif(value=='car_horn'):
    X[9].append(key)  
    random.shuffle(X[9])     

def Generate_Data(X,csvfilename,audio_train_directory,data_in_each_channel):

    df = pd.read_csv(csvfilename)
    Labels = list(df['class'].unique())
    Labels_int = list(range(len(Labels)))
    NUM_CLASSES = len(Labels_int)
    map_char_to_int = dict(zip(Labels,Labels_int))

    id_label = read_csv(csvfilename)
    NUM_CLASSES = len(X)

    lst = [0,1,2,3,4,5,6,7,8,9]
    X_data = []
    Y_data = []

    ################### Generating single channel data ####################
    n_channel = 1
    N_samples = data_in_each_channel[0]
    while(N_samples>0):
      for j in range(len(X)):
        if(N_samples>0):
          wav_file = []
          mul_label = []
          wav = random.choice(X[j])
          lab = id_label[wav]
          wav = audio_train_directory+"/"+wav
          wav_file.append(wav)
          mul_label.append(map_char_to_int[lab])
          x = wavs2feat(wav_file)
          y = encode(mul_label,5,NUM_CLASSES)
          X_data.append(x)
          Y_data.append(y)
          N_samples = N_samples-1
        else:
          break  
    ################### Generating 2 channel data #########################
    n_channel = 2
    N_samples = data_in_each_channel[1]
    while(N_samples>0):
      wav_file = []
      mul_label = []
      for i in range(len(X)):
        for j in range(i,len(X)):
          if(N_samples>0):
            wav1 = random.choice(X[i])
            lab1 = id_label[wav1]
            wav1 = audio_train_directory+"/"+wav1
            wav2 = random.choice(X[j])
            lab2 = id_label[wav2]
            wav2 = audio_train_directory+"/"+wav2
            wav_file.append(wav1)
            wav_file.append(wav2)
            mul_label.append(map_char_to_int[lab1])
            mul_label.append(map_char_to_int[lab2]) 
            x = wavs2feat(wav_file) 
            y = encode(mul_label,5,NUM_CLASSES)
            X_data.append(x)
            Y_data.append(y)
            wav_file = []
            mul_label = []
            N_samples = N_samples - 1
          else:
            break  
    ################## Generating 3 or more channel data ####################### 
    channel_size = [3,4,5]
    for l in range(len(channel_size)):
      n_channel = channel_size[l]
      N_samples = data_in_each_channel[l+2]
      lst_temp = list(np.copy(lst))
      for i in range(N_samples):
        wav_file = []
        mul_label = []
        for j in range(n_channel):
          k = random.choice(lst_temp)
          #lst_temp.remove(k)
          wav = random.choice(X[k])
          lab = id_label[wav]
          wav = audio_train_directory+"/"+wav
          wav_file.append(wav)
          mul_label.append(map_char_to_int[lab])
        x = wavs2feat(wav_file)  
        y = encode(mul_label,5,NUM_CLASSES)
        X_data.append(x)
        Y_data.append(y)

    temp = list(zip(X_data, Y_data)) 
    random.shuffle(temp) 
    X_data, Y_data = zip(*temp) 

    X_data = np.array(X_data)
    Y_data = np.array(Y_data)

    return X_data,Y_data,map_char_to_int    


In [ ]:
X_train = [[] for i in range(10)]
X_test = [[] for i in range(10)]

X_train[0] = X[0][0:150]
X_test[0] = X[0][150:]

X_train[1] = X[1][0:50]
X_test[1] = X[1][50:]

X_train[2] = X[2][0:140]
X_test[2] = X[2][140:]

X_train[3] = X[3][0:130]
X_test[3] = X[3][130:]

X_train[4] = X[4][0:180]
X_test[4] = X[4][180:]

X_train[5] = X[5][0:150]
X_test[5] = X[5][150:]

X_train[6] = X[6][0:150]
X_test[6] = X[6][150:]

X_train[7] = X[7][0:150]
X_test[7] = X[7][150:]

X_train[8] = X[8][0:150]
X_test[8] = X[8][150:]

X_train[9] = X[9][0:50]
X_test[9] = X[9][50:]

d1 = "/content/drive/MyDrive/project_specs_698/shared_train/labels_train.csv"
d2 = "/content/drive/MyDrive/project_specs_698/shared_train/audio_train"
X_train_data,Y_train_data,m = Generate_Data(X_train,d1,d2,[150,250,150,250,200])
X_test_data, Y_test_data,m = Generate_Data(X_test,d1,d2,[40,50,80,80,60])

In [ ]:
#y = np.array(y)
from sklearn.model_selection import train_test_split
X_train_data = X_train_data.reshape(X_train_data.shape[0],X_train_data.shape[1],X_train_data.shape[2],1)
X_test_data = X_test_data.reshape(X_test_data.shape[0],X_test_data.shape[1],X_test_data.shape[2],1)
#X_test,X_test,Y_test,Y_test = test_test_split(X_data,Y_data,test_size=0.2)

In [ ]:
def read_csv(filename):
    id_label = {}
    with open(filename,'r') as fid:
        for line in fid: # '176787-5-0-27.wav,engine_idling\n'
            tokens = line.strip().split(',') # ['176787-5-0-27.wav', 'engine_idling']
            id_label[tokens[0]] = tokens[1]
    return id_label

In [ ]:
from __future__ import print_function
import os
import numpy as np
import time
import sys
import matplotlib.pyplot as plot
from keras.layers import Bidirectional, TimeDistributed, Conv2D, MaxPooling2D, Input, GRU, Dense, Activation, Dropout, Reshape, Permute, MaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from sklearn.metrics import confusion_matrix

In [ ]:
def get_model(data_in, data_out, _cnn_nb_filt, _cnn_pool_size, _rnn_nb, _fc_nb,_fc_nb_pool_size):

     spec_start = Input(shape=(data_in.shape[-3], data_in.shape[-2], data_in.shape[-1]))
     spec_x = spec_start
     for _i, _cnt in enumerate(_cnn_pool_size):
        spec_x = Conv2D(filters=_cnn_nb_filt, kernel_size=(3, 3), padding='same')(spec_x)
        spec_x = BatchNormalization(axis=1)(spec_x)
        spec_x = Activation('relu')(spec_x)
        spec_x = MaxPooling2D(pool_size=(_cnn_pool_size[_i],1))(spec_x)
        spec_x = MaxPooling2D(pool_size=(1,_cnn_pool_size[_i]))(spec_x)
        spec_x = Dropout(dropout_rate)(spec_x)
     spec_x = Permute((2, 1, 3))(spec_x)
     spec_x = Reshape((spec_x.shape[-3], spec_x.shape[-2]*spec_x.shape[-1]))(spec_x)
     #spec_x = Permute((1,3,2))(spec_x)
     #spec_x = Reshape((5,-1))(spec_x)
     spex_x = GRU(units = 16,dropout=0.2, recurrent_dropout=0.2,return_sequences=True)(spec_x)

     for _f,_g in zip(_fc_nb,_fc_nb_pool_size):
        spec_x = TimeDistributed(Dense(_f))(spec_x)
        spec_x = MaxPooling1D(pool_size=(_g))(spec_x)
        spec_x = Dropout(dropout_rate)(spec_x)

     spec_x = TimeDistributed(Dense(data_out.shape[-1]))(spec_x)
     out = Activation('softmax', name='strong_out')(spec_x)

     _model = Model(inputs=spec_start, outputs=out)
     _model.compile(optimizer='Adam', loss='binary_crossentropy',metrics='accuracy')
     _model.summary()
     return _model

In [ ]:
cnn_nb_filt = 32          # CNN filter size
cnn_pool_size = [2, 2, 2]   # Maxpooling across frequency. Length of cnn_pool_size =  number of CNN layers
rnn_nb = [16, 16]           # Number of RNN nodes.  Length of rnn_nb =  number of RNN layers
fc_nb = [32,16,8] 
fc_nb_poolsize = [5,5,2]               # Number of FC nodes.  Length of fc_nb =  number of FC layers
dropout_rate = 0.2         # Dropout after each layer

model = get_model(X_train_data, Y_train_data, cnn_nb_filt, cnn_pool_size, rnn_nb, fc_nb,fc_nb_poolsize)
model1 = classifierModel(X_train_data[0].shape)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 2001, 1)]    0         
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 2001, 32)     320       
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 2001, 32)     512       
_________________________________________________________________
activation (Activation)      (None, 128, 2001, 32)     0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 2001, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 1000, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 64, 1000, 32)     

In [ ]:
history=model.fit(x=X_train_data, y=Y_train_data,validation_split = 0.1,verbose=2,epochs=100,shuffle = True,batch_size=32)

Epoch 1/100
29/29 - 9s - loss: 0.3144 - accuracy: 0.3182 - val_loss: 0.3070 - val_accuracy: 0.0680
Epoch 2/100
29/29 - 9s - loss: 0.2653 - accuracy: 0.3993 - val_loss: 0.3088 - val_accuracy: 0.0700
Epoch 3/100
29/29 - 9s - loss: 0.2499 - accuracy: 0.4224 - val_loss: 0.2948 - val_accuracy: 0.0680
Epoch 4/100
29/29 - 9s - loss: 0.2395 - accuracy: 0.4398 - val_loss: 0.2799 - val_accuracy: 0.3900
Epoch 5/100
29/29 - 9s - loss: 0.2331 - accuracy: 0.4578 - val_loss: 0.2792 - val_accuracy: 0.3940
Epoch 6/100
29/29 - 9s - loss: 0.2239 - accuracy: 0.4762 - val_loss: 0.2741 - val_accuracy: 0.3900
Epoch 7/100
29/29 - 9s - loss: 0.2184 - accuracy: 0.4862 - val_loss: 0.2772 - val_accuracy: 0.3900
Epoch 8/100
29/29 - 9s - loss: 0.2186 - accuracy: 0.4938 - val_loss: 0.2664 - val_accuracy: 0.3880
Epoch 9/100
29/29 - 9s - loss: 0.2170 - accuracy: 0.4976 - val_loss: 0.2737 - val_accuracy: 0.4180
Epoch 10/100
29/29 - 9s - loss: 0.2121 - accuracy: 0.5153 - val_loss: 0.2563 - val_accuracy: 0.4140
Epoch 11/

In [ ]:
model.evaluate(X_test_data,Y_test_data)

10/10 [==============================] - 1s 108ms/step - loss: 0.1475 - accuracy: 0.7284


[0.14754721522331238, 0.7283871173858643]

In [ ]:
model.save("/content/drive/MyDrive/My_model(2)")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/drive/MyDrive/My_model(2)/assets


In [ ]:
a = model.predict(X_test_data)#sample check

In [ ]:
np.argmax(a[78],axis=1)

array([ 2,  7,  1, 10, 10])

In [ ]:
np.argmax(Y_test_data[78],axis=1)

array([ 2,  7,  1, 10, 10])

In [ ]:
def func(label,m):
  char_lst = []
  for i in range(label.size):
    if(label[0][i]==10):
      break
    for key in m.keys():
      if(m[key]==label[0][i]):
        char_lst.append(key)
        break
  new_lst = []
  previous_value = None

  for elem in char_lst:
    if elem!=previous_value:
      new_lst.append(elem)
      previous_value = elem

  label_char = '-'.join(new_lst)  

  return label_char  




In [ ]:
####################### EVALUATION ON TEST SET ####################################

directory = "/content/drive/MyDrive/test_task2/feats"
direc = os.fsencode(directory)
lis_dir = {}
for np_fil in os.listdir(direc):
  fil = os.fsdecode(np_fil)
  X = process(np.load(directory+"/"+fil))
  #print(X.shape)
  label = np.argmax(model.predict(np.array([X])),axis=2)
  #print(label.shape)
  label_char = func(label,m)
  lis_dir[fil] = label_char

In [ ]:
lis_dir

{'a001.npy': 'siren-street_music-siren',
 'a002.npy': 'street_music-car_horn-siren',
 'a003.npy': 'siren-street_music-children_playing',
 'a004.npy': 'siren-children_playing-dog_bark',
 'a005.npy': 'air_conditioner-children_playing',
 'a006.npy': 'jackhammer-engine_idling-jackhammer-air_conditioner',
 'a007.npy': 'siren-jackhammer',
 'a008.npy': 'jackhammer',
 'a009.npy': 'dog_bark-air_conditioner-children_playing',
 'a010.npy': 'engine_idling-drilling',
 'a011.npy': 'drilling-dog_bark-jackhammer',
 'a012.npy': 'car_horn-jackhammer-street_music',
 'a013.npy': 'children_playing-drilling',
 'a014.npy': 'engine_idling-jackhammer',
 'a015.npy': 'jackhammer-children_playing-dog_bark-drilling',
 'a016.npy': 'children_playing-jackhammer-drilling',
 'a017.npy': 'children_playing-jackhammer',
 'a018.npy': 'jackhammer-siren',
 'a019.npy': 'jackhammer-air_conditioner',
 'a020.npy': 'jackhammer-engine_idling-children_playing-street_music',
 'a021.npy': 'siren-car_horn-gun_shot',
 'a022.npy': 'stre

In [ ]:
########################## Converting to .csv ##################################

import csv
field_names = ['file_name','Predicted_label']
with open('test.csv','w') as f:
  csv.DictWriter(f,fieldnames = field_names)
  for key in lis_dir.keys():
    f.write("%s,%s\n"%(key,lis_dir[key]))

In [ ]:
df = pd.read_csv("test.csv")